In [1]:
import pandas as pd
import altair as alt
import colorcet as cc

In [2]:
mapping = pd.read_csv('../../02_exomiser_manuscript/GS_ID_mapping.csv')
p_values = [1, 0.5, 0.3, 0.2, 0.1, 0.05, 0.01]
_range = cc.glasbey_light[::5][:7]
_range


['#d60000', '#ffa52f', '#93ac83', '#c86e66', '#f4bfb1', '#56642a', '#8c9ab1']

In [3]:
def create_pval_plot(success_table,optimized_run_type, p_values=[0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1], _range=cc.glasbey_light[::5][:7][::-1]):
    p_df=[]
    for p in p_values:
        p_filt = success_table[success_table['Variant_Level_noMOI_p_' + str(optimized_run_type)]<=p]
        print(p, len(p_filt),len(p_filt)/len(success_table))
        p_df.append(['P_filt',p, len(p_filt), (len(p_filt)/len(success_table))*100])
    p_df = pd.DataFrame( p_df, columns = ['Run_Type', 'P','Num_Variants','Percent_Variants'])


    pval_plot = alt.Chart(p_df, title='P-Value filtering').mark_circle(size=100, filled=True, stroke='black',opacity=1).encode(
        x=alt.X('Percent_Variants',title='Percent of Causal Variants ≤ Exomiser p-value' ,scale=alt.Scale(domain=[0,100])),
        y= alt.Y('Run_Type'),
        color=alt.Color('P:N', sort='-x', scale=alt.Scale(domain=p_values, range=_range), legend=alt.Legend(orient='top',direction='horizontal')),
        tooltip = ['Run_Type', 'Num_Variants','Percent_Variants', 'P']
    ).properties(
        width=400,
        height=50)
    return pval_plot



def create_box_plot(input_file, success_table,p_values=[1, 0.5, 0.3, 0.2, 0.1, 0.05, 0.01], _range=cc.glasbey_light[::5][:7]):
    ids = [x.split('_')[0] for x in list(success_table['ID'])]
    data=[]
    for p in p_values:
        results = input_file[input_file['P-VALUE'] < p]
        for ID in set(ids):
            num_genes = len(set(results[results['Patient_ID']==ID]['GENE_SYMBOL']))
            num_variants = len(set(results[results['Patient_ID']==ID]['variant']))


            data.append([ID, p, num_genes, num_variants])
    df = pd.DataFrame(data, columns = ['ID', 'Threshold', 'Num_Genes', 'Num_Variants'])

    pval_boxplot = alt.Chart(df,width=alt.Step(100)).mark_boxplot().encode(
        alt.X("Num_Variants:Q", title='Number of Variants Prioritized'),#,scale=alt.Scale(domain=[0,1200])),
        alt.Y("Threshold:N",title='Max p-value', sort=p_values),
        color = alt.Color('Threshold:N',sort=p_values, scale=alt.Scale(domain=p_values,range=_range), legend=None),
        #tooltip = ['ID','numUniqueRankedGenes', 'Family_Type', 'RunType'],
        #yOffset=alt.YOffset('RunType', sort=RUN_TYPES, bandPosition=0)
    ).properties(height=150, width=400)

    return pval_boxplot




## WGS Exomiser (A-B)

In [6]:
gs_exomiser_success_table = pd.read_csv("../../02_exomiser_manuscript/fig_06_supp_fig_16/GS_exomiser_fig4_input_12.16.24.tsv", sep='\t')
##
gs_exomiser_pvals = create_pval_plot(gs_exomiser_success_table, 'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL')
##number of genes ranked based on exomiser p
gs_exomiser_input =pd.read_csv('../../02_exomiser_manuscript/supp_fig_13/GS_exomiser_all_results_optimized.tsv', sep='\t')
split_id = []
for id in gs_exomiser_input['ID']:
    split_id.append(id.split('_')[0])

gs_exomiser_input['variant'] = split_id
print(len(set(gs_exomiser_input['ID'])))
print(len(set(gs_exomiser_input['variant'])))

gs_exomiser_box_plot  = create_box_plot(gs_exomiser_input, gs_exomiser_success_table)


0.01 173 0.5844594594594594
0.05 226 0.7635135135135135
0.1 247 0.8344594594594594
0.2 273 0.9222972972972973
0.3 283 0.956081081081081
0.5 286 0.9662162162162162
1 288 0.972972972972973
35557
31442


In [7]:
gs_exomiser = alt.hconcat(gs_exomiser_pvals, gs_exomiser_box_plot).resolve_scale(color='independent')
gs_exomiser

alt.HConcatChart(...)

## WGS Genomiser

In [9]:
gs_genomiser_success_table = pd.read_csv("../../02_exomiser_manuscript/fig_06_supp_fig_16/GS_genomiser_fig4_input_12.16.24.tsv", sep='\t')
##
gs_genomiser_pvals = create_pval_plot(gs_genomiser_success_table, 'noN_filtered_15_18_human_REMM_revel_mvp_alphaM_spliceAI_PSF_501_reg_filter_noWL_genomiser')
##number of genes ranked based on exomiser p
gs_genomiser_input =pd.read_csv('../../02_exomiser_manuscript/supp_fig_13/GS_genomiser_all_results_optimized.tsv', sep='\t')

split_id = []
for id in gs_genomiser_input['ID']:
    split_id.append(id.split('_')[0])

gs_genomiser_input['variant'] = split_id
print(len(set(gs_genomiser_input['ID'])))
print(len(set(gs_genomiser_input['variant'])))


gs_genomiser_box_bplot = create_box_plot(gs_genomiser_input, gs_genomiser_success_table)

0.01 21 0.35
0.05 36 0.6
0.1 40 0.6666666666666666
0.2 41 0.6833333333333333
0.3 41 0.6833333333333333
0.5 41 0.6833333333333333
1 41 0.6833333333333333
56835
53145


In [10]:
gs_genomiser = alt.hconcat(gs_genomiser_pvals, gs_genomiser_box_bplot).resolve_scale(color='independent')
gs_genomiser

alt.HConcatChart(...)

## WES Exomiser

In [16]:
es_exomiser_success_table = pd.read_csv("../../02_exomiser_manuscript/fig_03/ES_genomiser_fig3_input_1.13.25.tsv", sep='\t')
##
es_exomiser_pvals = create_pval_plot(es_exomiser_success_table, 'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL')
##number of genes ranked based on exomiser p
es_exomiser_input =pd.read_csv('../../02_exomiser_manuscript/supp_fig_13/es_exomiser_all_results_optimized.tsv', sep='\t')


split_id = []
for id in es_exomiser_input['ID']:
    split_id.append(id.split('_')[0])

es_exomiser_input['variant'] = split_id
print(len(set(es_exomiser_input['ID'])))
print(len(set(es_exomiser_input['variant'])))



es_exomiser_box_bplot = create_box_plot(es_exomiser_input, es_exomiser_success_table)

0.01 118 0.7712418300653595
0.05 130 0.8496732026143791
0.1 134 0.8758169934640523
0.2 144 0.9411764705882353
0.3 144 0.9411764705882353
0.5 146 0.954248366013072
1 148 0.9673202614379085
14978
13520


In [17]:
es_exomiser = alt.hconcat(es_exomiser_pvals, es_exomiser_box_bplot).resolve_scale(color='independent')
es_exomiser

alt.HConcatChart(...)

In [18]:
alt.vconcat(gs_exomiser, gs_genomiser, es_exomiser).resolve_scale(color='shared').configure_axis(
    labelPadding= 5,
    labelLimit=0,
    labelFontSize=15, 
    titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
        labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial')

alt.VConcatChart(...)